# Battle of Neighbourhoods

### Concept

Every 90 seconds a 2 children go missing somewhere in the US. and GPS devices need human interaction. If guardian is not notified 

### The Problem:

given the coordinates of a child , predict whether he/she is lost/kidnapped or safe.

This idea is basically,assume there is an alone child, the child has a GPS device that sends it coordinates to a server regularly. The idea is to create a script run on the server that is able to analyse the coordinates and determine whether he is lost (or  kidnapped) or in a safe area. I will use Foursquare API to see the available venues in 1km radius. If there is None or the available venues are not child related (e.g. bars, ..etc). the script should assume the kid is kidnapped or lost

### Target Audience

12-year-old children guardians

Alzhiemer's guardians

### Businesses that might be interested

Businesses that focus on children safety products




## Data

#### Foursquare API

Input: coordinates , Output: Venues in 1 km

#### A generated sqlite3 database of coordinates

A database of coordinates and time, to test the script, most of them are coordinates in the middle of the desert, the ocean, the sea. only one of them is actually a place  where a child can be

#### A JSON file i written with categories of Venues that are unlikely to be visited by a lonely child 


In [1]:
import numpy as np
import pandas as pd
import sqlite3
import requests
import json
import random
import matplotlib.pyplot as plt
import folium # map rendering library
from datetime import date

Now we read the sqlite3 database

In [2]:
con = sqlite3.connect("points.db")
cursor = con.cursor()

print("Database created and Successfully Connected to SQLite")

Database created and Successfully Connected to SQLite


In [3]:
cursor.execute("SELECT * FROM log")

In [4]:
coordsDB = cursor.fetchall()

In [5]:
cursor.close()
con.close()

print("Closed SQLite3 connection")

Closed SQLite3 connection


In [6]:
coords = pd.DataFrame(coordsDB,columns=["Longitude","Latitude","Date","Latency"])

In [7]:
coords.reset_index()
coords = coords.drop(labels=["Latency"],axis=1)
coords.head()

,Longitude,Latitude,Date
0,0.0,0.0,2020-07-27T17:19:40.551Z
1,1.0,1.0,2020-07-27T17:19:42.572Z
2,2.0,2.0,2020-07-27T17:19:44.584Z
3,3.0,3.0,2020-07-27T17:19:46.595Z
4,4.0,4.0,2020-07-27T17:19:48.604Z


In [8]:
client_id="4YCXG4R5QPPQP2AMT2PVQYKEBKXJTCR2CCAGPD5SBGW4ZHPF"
client_secret="URRAO4CB4GCLB1YDICP5AXKW1Z4FO1BMLY5PCKMGG4CE2L24"
version = str(date.today()).replace("-","")

In [9]:
with open("categories.json") as f:
    nonChild = json.load(f)['nonChild']

In [10]:
def getVenues(Longitudes,Latitudes):
    RADIUS = 1000 # Getting in 1km radius
    SafeVenues = []
    for (lat,lng) in zip(Longitudes,Latitudes):
        url =  f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&ll={lat},{lng}&radius={RADIUS}'
        res = requests.get(url).json()["response"]
        availVenues = []
        try:
            for v in res['groups'][0]['items']:
                for cat in v['venue']['categories']:
                    if cat not in nonChild:
                        availVenues.append(cat)
        
            SafeVenues.append(len(availVenues))
        except KeyError:
            SafeVenues.append(0)
    return SafeVenues


Now we will add a new column with the number of safe venues in each location

In [11]:
coords['SafeVenues'] = getVenues(coords['Longitude'],coords['Latitude'])
coords.head()

,Longitude,Latitude,Date,SafeVenues
0,0.0,0.0,2020-07-27T17:19:40.551Z,0
1,1.0,1.0,2020-07-27T17:19:42.572Z,2
2,2.0,2.0,2020-07-27T17:19:44.584Z,0
3,3.0,3.0,2020-07-27T17:19:46.595Z,0
4,4.0,4.0,2020-07-27T17:19:48.604Z,0


Now we have each coordinate with the Safe Venues available

In [12]:
coords

,Longitude,Latitude,Date,SafeVenues
0,0.00000,0.00000,2020-07-27T17:19:40.551Z,0
1,1.00000,1.00000,2020-07-27T17:19:42.572Z,2
2,2.00000,2.00000,2020-07-27T17:19:44.584Z,0
3,3.00000,3.00000,2020-07-27T17:19:46.595Z,0
4,4.00000,4.00000,2020-07-27T17:19:48.604Z,0
5,5.00000,5.00000,2020-07-27T17:19:50.615Z,0
6,6.00000,6.00000,2020-07-27T17:19:52.626Z,0
7,7.00000,7.00000,2020-07-27T17:19:54.637Z,0
8,8.00000,8.00000,2020-07-27T17:19:56.647Z,0
9,9.00000,9.00000,2020-07-27T17:19:58.659Z,0


Normalising

In [14]:
coords['howSafe'] = (coords['SafeVenues']/coords['SafeVenues'].max())*100
coords

,Longitude,Latitude,Date,SafeVenues,howSafe
0,0.00000,0.00000,2020-07-27T17:19:40.551Z,0,0.000000
1,1.00000,1.00000,2020-07-27T17:19:42.572Z,2,6.666667
2,2.00000,2.00000,2020-07-27T17:19:44.584Z,0,0.000000
3,3.00000,3.00000,2020-07-27T17:19:46.595Z,0,0.000000
4,4.00000,4.00000,2020-07-27T17:19:48.604Z,0,0.000000
5,5.00000,5.00000,2020-07-27T17:19:50.615Z,0,0.000000
6,6.00000,6.00000,2020-07-27T17:19:52.626Z,0,0.000000
7,7.00000,7.00000,2020-07-27T17:19:54.637Z,0,0.000000
8,8.00000,8.00000,2020-07-27T17:19:56.647Z,0,0.000000
9,9.00000,9.00000,2020-07-27T17:19:58.659Z,0,0.000000
